In [94]:
import os

In [95]:
%pwd #present working directory

'C:\\Users\\mhossain\\Documents\\Machine_Learning_Projects\\kidney_diseases'

In [96]:
os.chdir("C:/Users/mhossain/Documents/Machine_Learning_Projects/kidney_diseases")

In [97]:
#I need to go back the project directory
#os.chdir("../") #change the working directory

In [98]:
%pwd

'C:\\Users\\mhossain\\Documents\\Machine_Learning_Projects\\kidney_diseases'

# Entity 
* return type of any function
* return type of my data ingestion function

In [99]:
from dataclasses import dataclass
from pathlib import Path

In [116]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path #these are the access of variable from the other class
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [117]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [118]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, #data source and destination
                 params_filepath =PARAMS_FILE_PATH): #hyperparameter
        
        self.config=read_yaml(config_filepath) #read data by yaml
        self.params=read_yaml(params_filepath) #read hyperparameter by yaml
    
        create_directories([self.config.artifacts_root]) #create artifacts folder
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion #taking input from config file as source data
        
        create_directories([config.root_dir])#??
        
        data_ingestion_config = DataIngestionConfig( #copy data from DataIngestionConfig
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

# src/components

In [119]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [120]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config
        
    def download_file(self)->str:
        '''
        Fetch data from the url
        
        '''

        try:
            dataset_url=self.config.source_URL #data source url
            zip_download_dir=self.config.local_data_file #download local file
            os.makedirs("artifacts/data_ingestion", exist_ok=True) #create folder
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id=dataset_url.split("/")[-2]
            prefix="https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """
        
        unzip_path=self.config.unzip_dir #unzip path
        os.makedirs(unzip_path,exist_ok=True) #create unzip folder
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 

# Pipeline

* execution command one after another

In [121]:
try:
    config=ConfigurationManager() #
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-02-03 15:21:49,832:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-02-03 15:21:49,834:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-03 15:21:49,834:INFO:common:created directory at: artifacts]
[2024-02-03 15:21:49,835:INFO:common:created directory at: artifacts/data_ingestion]
[2024-02-03 15:21:49,836:INFO:4141667296:Downloading data from https://drive.google.com/file/d/13uWZZ1nhijwqJfg4hCvgNfpm7RSLi9LM/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=13uWZZ1nhijwqJfg4hCvgNfpm7RSLi9LM
From (redirected): https://drive.google.com/uc?/export=download&id=13uWZZ1nhijwqJfg4hCvgNfpm7RSLi9LM&confirm=t&uuid=2186f05b-2b64-4beb-9cc2-ff350a1147d9
To: C:\Users\mhossain\Documents\Machine_Learning_Projects\kidney_diseases\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [02:35<00:00, 6.05MB/s] 

[2024-02-03 15:24:26,995:INFO:4141667296:Downloaded data from https://drive.google.com/file/d/13uWZZ1nhijwqJfg4hCvgNfpm7RSLi9LM/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [ ]:
%pwd

'C:\\Users\\mhossain\\Documents\\Machine_Learning_Projects\\kidney_diseases'